# LangChain + kimi 实现对话机器人

## LLMChain 实现单轮对话

In [30]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
import logging
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.language_models import LLM
from openai import OpenAI
from langchain.prompts import PromptTemplate

In [31]:
class Kimi(LLM):

    #llm 属性可不定义
    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "kimillm"
        
    #必须定义_call方法
    def _call(self, prompt: str, **kwargs: any) -> str:
    
        try:
          
            client = OpenAI(
                #此处请替换自己的api
                api_key="sk-IfM9SFhekY2FJhNkQu9ieDyzrVIulWEHl0p481QAFTaBQLVS",
                base_url="https://api.moonshot.cn/v1",
            )
            completion = client.chat.completions.create(
    model="moonshot-v1-8k",
    messages=[
        # 系统消息，定义了Kimi的角色和行为准则。
        {
            "role": "system",
            "content": "你是 Kimi，你现在是一名肯德基的服务生。",
        },
    
        {"role": "user", "content": prompt},
    ],
    temperature=0.5,
    stream=False,
)
            return completion.choices[0].message.content
        except Exception as e:
            logging.error(f"Error in Kimi _call: {e}", exc_info=True)
            raise


In [32]:
# 实例化chat大模型
llm = Kimi()

In [33]:
# 提示词模板
template = """{question}"""
prompt = PromptTemplate(template=template, input_variables=["question"])

In [34]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [35]:
llm_chain.run("今天星期四，请问有什么点单推荐吗？")

'当然，很高兴为您推荐一些肯德基的热门餐点：\n\n1. **经典炸鸡套餐**：包含多块新鲜炸鸡，搭配薯条和可乐，是肯德基的招牌选择。\n\n2. **香辣鸡腿堡**：多汁的鸡腿肉搭配香辣酱料，是喜欢辣味的您的不二之选。\n\n3. **奥尔良烤翅**：独特的奥尔良风味，外皮酥脆，内肉鲜嫩，是很多人的心头好。\n\n4. **早餐系列**：如果您在早餐时间点单，可以尝试肯德基的早餐系列，如帕尼尼、粥品等。\n\n5. **健康轻食**：如果您在寻找更健康的选择，可以考虑沙拉、烤鸡胸肉等轻食选项。\n\n6. **甜品和小吃**：餐后不妨来一份冰淇淋或蛋挞，为用餐画上完美的句号。\n\n7. **特惠套餐**：通常肯德基会有特惠套餐，性价比较高，可以根据您的需求选择。\n\n8. **季节限定产品**：肯德基会根据季节推出限定产品，可以询问店员了解最新的限定美味。\n\n您可以根据自己的口味和需求选择，或者告诉我您的偏好，我可以为您提供更个性化的推荐。'

**如果要实现常规的对话，需要怎么做？**

## 带记忆功能的聊天对话（Conversation with Memory）

In [37]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [38]:
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

In [39]:
conversation.predict(input="什么汉堡比较好吃？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 什么汉堡比较好吃？
AI:

> Finished chain.


'AI: 肯德基有很多美味的汉堡，比如香辣鸡腿堡、新奥尔良烤鸡腿堡、劲脆鸡腿堡等。香辣鸡腿堡以辣味著称，新奥尔良烤鸡腿堡带有独特的甜辣风味，劲脆鸡腿堡则以酥脆的鸡腿肉和多汁的生菜为特色。每个人的口味不同，你可以根据个人口味来选择。如果你喜欢辣味，可以尝试香辣鸡腿堡；如果你喜欢烤味，可以选择新奥尔良烤鸡腿堡。当然，你也可以尝试其他口味的汉堡，找到最适合自己的那一款。'

In [40]:
conversation.predict(input="饮料有没有推荐呢？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 什么汉堡比较好吃？
AI: AI: 肯德基有很多美味的汉堡，比如香辣鸡腿堡、新奥尔良烤鸡腿堡、劲脆鸡腿堡等。香辣鸡腿堡以辣味著称，新奥尔良烤鸡腿堡带有独特的甜辣风味，劲脆鸡腿堡则以酥脆的鸡腿肉和多汁的生菜为特色。每个人的口味不同，你可以根据个人口味来选择。如果你喜欢辣味，可以尝试香辣鸡腿堡；如果你喜欢烤味，可以选择新奥尔良烤鸡腿堡。当然，你也可以尝试其他口味的汉堡，找到最适合自己的那一款。
Human: 饮料有没有推荐呢？
AI:

> Finished chain.


'肯德基提供多种饮料选择，以满足不同顾客的口味需求。如果你喜欢碳酸饮料，可以选择经典的百事可乐，它与汉堡搭配起来非常合适。如果你想要一些更健康的选择，可以考虑新鲜制作的果汁，比如橙汁或苹果汁，它们既清新又解渴。此外，肯德基还有咖啡、热茶等热饮供你选择。如果你在寻找一些特别的口味，肯德基的冰沙系列也是不错的选择，它们通常口感丰富，适合炎热的天气。当然，你也可以根据个人口味和当天的心情来选择饮料，享受一顿美味的快餐。'

In [41]:
conversation.predict(input="我还想要甜品。")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 什么汉堡比较好吃？
AI: AI: 肯德基有很多美味的汉堡，比如香辣鸡腿堡、新奥尔良烤鸡腿堡、劲脆鸡腿堡等。香辣鸡腿堡以辣味著称，新奥尔良烤鸡腿堡带有独特的甜辣风味，劲脆鸡腿堡则以酥脆的鸡腿肉和多汁的生菜为特色。每个人的口味不同，你可以根据个人口味来选择。如果你喜欢辣味，可以尝试香辣鸡腿堡；如果你喜欢烤味，可以选择新奥尔良烤鸡腿堡。当然，你也可以尝试其他口味的汉堡，找到最适合自己的那一款。
Human: 饮料有没有推荐呢？
AI: 肯德基提供多种饮料选择，以满足不同顾客的口味需求。如果你喜欢碳酸饮料，可以选择经典的百事可乐，它与汉堡搭配起来非常合适。如果你想要一些更健康的选择，可以考虑新鲜制作的果汁，比如橙汁或苹果汁，它们既清新又解渴。此外，肯德基还有咖啡、热茶等热饮供你选择。如果你在寻找一些特别的口味，肯德基的冰沙系列也是不错的选择，它们通常口感丰富，适合炎热的天气。当然，你也可以根据个人口味和当天的心情来选择饮料，享受一顿美味的快餐。
Human: 我还想要甜品。
AI:

> Finished chain.


'肯德基的甜品种类也很丰富，可以为你的餐点增添甜蜜。以下是一些受欢迎的甜品推荐：\n\n1. 圣代：肯德基的圣代有多种口味，如草莓、巧克力、香草等，上面通常会撒上一些坚果碎，口感丰富。\n\n2. 蛋挞：肯德基的蛋挞外皮酥脆，内馅香甜滑嫩，是很多人的心头好。\n\n3. 冰淇淋：肯德基提供多种口味的冰淇淋，包括原味、巧克力、草莓等，冰凉爽口。\n\n4. 红豆派：红豆派外皮酥脆，内馅是甜而不腻的红豆沙，很适合喜欢红豆口味的顾客。\n\n5. 巧克力蛋糕：肯德基的巧克力蛋糕口感细腻，巧克力味道浓郁，是巧克力爱好者的好选择。\n\n你可以根据自己的喜好和当天的心情来选择甜品，享受一顿完美的肯德基快餐。甜品种类会根据季节和地区有所不同，建议到店查看最新的甜品菜单。'